<a href="https://colab.research.google.com/github/alfazick/AppliedLLMCourse/blob/main/AppliedLLMModule1Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Fine Grained Inference Options

# small model, to run inference on cpu
# model.eval() and torch.no_grad()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device).eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
model.num_parameters()

494032768

In [ ]:
# concept and shape explanation

# 1) Tokenize -> Forward Pass| Logits -> "Last Prediction" ->[Softmax] and "Do what you want"

prompt = "Where McAllen is located"
inputs = tokenizer(prompt,return_tensors="pt")

print(inputs)

for token in inputs["input_ids"]:
    print(tokenizer.decode(token),sep = "")

{'input_ids': tensor([[ 9064,  4483, 79877,   374,  7407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
Where McAllen is located


In [ ]:
# or for better visualisation

prompt = "Where McAllen is located"
inputs = tokenizer(prompt, return_tensors="pt")

print(inputs)
print(f"Shape: {inputs['input_ids'].shape}")  # [1, 5]
print("\nIndividual tokens:")

for i, token_id in enumerate(inputs["input_ids"][0]):  # [0] to get first batch item
    decoded = tokenizer.decode([token_id])  # decode single token
    print(f"Position {i}: token_id={token_id.item():5d} → '{decoded}'")

{'input_ids': tensor([[ 9064,  4483, 79877,   374,  7407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
Shape: torch.Size([1, 5])

Individual tokens:
Position 0: token_id= 9064 → 'Where'
Position 1: token_id= 4483 → ' Mc'
Position 2: token_id=79877 → 'Allen'
Position 3: token_id=  374 → ' is'
Position 4: token_id= 7407 → ' located'


In [ ]:
# so we have [1,5] or [B,T]
# where B is a batch size (how many sequences process at once)
# where T is a sequence length (number of tokens in your prompt)


# let's focus on single prompt then we expand our example
# so we pass our inputs into model and model will return outputs with
# prob for each token position

# [B,T,V] where V is a vocabulary size, Embedding(151936, 896)

# or V is every possible token the model knows


In [ ]:
with torch.no_grad():
    out = model(**inputs)
    logits = out.logits

print(logits.shape)

# so please notice model produces probabilities for each token in the sequence
# including the one you provided yourself

torch.Size([1, 5, 151936])


In [ ]:
# so since we are intersted into prediction of next token,
# we will access the prediction for the last token

# so notice afer we extract last token predictions we will get [1,V], because
# we are "predicting"

logits_last = out.logits[:,-1,:] # so we are saying in first dimension all ,
# even though techincally we have only one,
# in second last from the end
# and in third obviously all predictions


print(logits_last.shape)


torch.Size([1, 151936])


In [ ]:
logits_last[:10] # notice it's not really a probability it's raw logits output
# and also notice it's random 10 token
# most likely special characters

tensor([[13.0130,  8.7147,  8.5017,  ..., -3.1183, -3.1183, -3.1184]])

In [ ]:
for i in range(10):
    print(tokenizer.decode(i))

!
"
#
$
%
&
'
(
)
*


In [ ]:
# ok so basically since we are playing a probability game for next token prediction
# let see the most 5 most likely tokens our model predicting next

# Raw version 1

top_k = 5
top_logits,top_indices_logits = torch.topk(logits_last, k = top_k, dim= -1)

In [ ]:
top_logits,top_indices_logits

(tensor([[19.6082, 18.2642, 18.2444, 17.3177, 17.1950]]),
 tensor([[ 11, 304, 374, 389, 198]]))

In [ ]:
# so let's do manualy
manual_logits = logits_last[-1].tolist() # remove batch dimension convert to list Converts PyTorch tensor to Python list

len(manual_logits)

151936

In [ ]:
logits_with_idx = [(idx,value) for idx,value in enumerate(manual_logits)]


In [ ]:
logits_sorted = sorted(logits_with_idx, key = lambda x:x[1]) # sort by value

In [ ]:
logits_sorted[-5:]

[(198, 17.194992065429688),
 (389, 17.317724227905273),
 (374, 18.244359970092773),
 (304, 18.264217376708984),
 (11, 19.608192443847656)]

In [ ]:
# so not surprisingly we have same top_k probabilities

In [ ]:
# let's see what are tokens value

for token_idx,value in logits_sorted[-5:][::-1]:
    print(token_idx,tokenizer.decode([token_idx]),"======== > value",value)

11 , ======== > value 19.608192443847656
304  in ======== > value 18.264217376708984
374  is ======== > value 18.244359970092773
389  on ======== > value 17.317724227905273
198 
 ======== > value 17.194992065429688


In [ ]:

for i in range(top_k):
    token_id = top_indices_logits[0, i].item()
    logit_value = top_logits[0, i].item()
    token_text = tokenizer.decode([token_id])

    print(f"{i+1}. Token ID: {token_id:6d} | Logit: {logit_value:8.3f} | Text: '{token_text}'")


1. Token ID:     11 | Logit:   19.608 | Text: ','
2. Token ID:    304 | Logit:   18.264 | Text: ' in'
3. Token ID:    374 | Logit:   18.244 | Text: ' is'
4. Token ID:    389 | Logit:   17.318 | Text: ' on'
5. Token ID:    198 | Logit:   17.195 | Text: '
'


In [ ]:
# ok so it's very hard to interpret this values as probability, and apply safely
# safely techniques we are about to see
# so the common procedure is to apply softmax  and get aka "probability"
# fit nicely with cross-entropy loss function
# https://www.youtube.com/watch?v=6ArSys5qHAU
# as well softmax is a convinient activation function for backpropagation


In [ ]:
import torch.nn.functional as F

logits_last = out.logits[:,-1,:]

probs = F.softmax(logits_last,dim=-1)
top_probs, top_indices_probs = torch.topk(probs, k=top_k, dim=-1)

for i in range(top_k):
    token_id = top_indices_probs[0, i].item()
    probability = top_probs[0, i].item()
    token_text = tokenizer.decode([token_id])

    print(f"{i+1}. Token ID: {token_id:6d} | Prob: {probability:.4f} ({probability*100:.2f}%) | Text: '{token_text}'")

print("\n" + "=" * 60)
print(f"✓ Total probability sums to: {probs.sum().item():.6f}")
print("=" * 60)


1. Token ID:     11 | Prob: 0.4210 (42.10%) | Text: ','
2. Token ID:    304 | Prob: 0.1098 (10.98%) | Text: ' in'
3. Token ID:    374 | Prob: 0.1077 (10.77%) | Text: ' is'
4. Token ID:    389 | Prob: 0.0426 (4.26%) | Text: ' on'
5. Token ID:    198 | Prob: 0.0377 (3.77%) | Text: '
'

✓ Total probability sums to: 1.000071


In [ ]:
# so top 5 tokens are same in both versions(softmax preserves the order)

# what is good about softmax values are between 0 and 1 and sum 1.0
# so easy to interpret as probability
# though it does much more

# For fun to watch about softmax :))
# https://www.youtube.com/watch?v=KpKog-L9veg
# https://www.youtube.com/watch?v=ytbYRIN0N4g

In [ ]:
# ok so now let's fork into two ideas
# 1) generation till completion
# 2) batch processing
